In [13]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import keras.backend
if keras.backend.backend() != 'tensorflow':
    raise BaseException("This script uses other backend")
else:
    keras.backend.set_image_dim_ordering('th')
    print("Backend ok")

from pythainlp.tokenize import word_tokenize
from pythainlp.word_vector.thai2vec import get_model
from numpy import zeros, array
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from pandas import read_csv
from re import compile
import pickle

word2vec = get_model()

Using TensorFlow backend.


Backend ok


In [636]:
def clean(sentence):
    return sentence.strip()

def remove_tags(sentence):
    regex = compile(r'<\/?[\w-]+>')
    return regex.sub('', sentence)

def remove_dash(sentence):
    regex = compile(r'-')
    return regex.sub('', sentence)

def get_word2idx(tokenized_sentences):
    word2idx ={}
    for sentence in tokenized_sentences:
        for word in sentence:
            if word not in word2idx:
                word2idx[word] = len(word2idx) + 1
    word2idx['UNK'] = len(word2idx)
    return word2idx

def get_index(word, word2idx):
    return word2idx[word] if word in word2idx else word2idx['UNK']

def get_embeddings(word2vec, word2idx, dim=300):
    embeddings = zeros((len(word2idx), dim))
    for (word, i) in word2idx.items():
        if word != 'UNK' and word in word2vec.index2word:
            embeddings[i] = word2vec.word_vec(word)
        else:
            pass
    return embeddings

def get_training_data(tokenized_sentences, classes, classes2, word2idx, maxlen=1000, class_num=4, class_num_two=4):
    train_x = array([[get_index(word, word2idx) for word in sentence] for sentence in tokenized_sentences])
    train_y = [cls for cls in classes]
    train_z = [cls for cls in classes2]

    train_x = pad_sequences(train_x, maxlen=maxlen, dtype='int32', padding='post', truncating='pre', value=0.)
    train_y = array([[1 if i == cls else 0 for i in range(0, class_num)] for cls in train_y])
    train_z = array([[1 if i == cls else 0 for i in range(0, class_num_two)] for cls in train_z])

    return (train_x, train_y, train_z)

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    r = {}
    with open(filename, 'rb') as f:
        r = pickle.load(f)
    return r

In [637]:
from sklearn.model_selection import train_test_split
import numpy as np
mood = []
genre = []


def get_corpus(path):
    global mood, genre
    corpus = read_csv(path)
    corpus = corpus[['sentence', 'mood', 'genre']]
    corpus['sentence'] = corpus['sentence'].apply(clean).apply(remove_tags)
    corpus['tokenized_sentence'] = corpus['sentence'].apply(word_tokenize, engine='deepcut')
    corpus['genre'] = corpus['genre'].apply(remove_dash)
    mood += list(corpus.mood.unique())
    genre += list(corpus.genre.unique())
    return corpus

corpus = get_corpus('kuy.csv')
train_corpus = corpus.copy()
test_corpus = corpus.sample(n=7,replace=True).copy()


mood = sorted(list(set(mood)), reverse=True)
genre = sorted(list(set(genre)), reverse=True)
mood2idx = dict(zip(mood, [i for i in range(len(mood))]))
genre2idx = dict(zip(genre, [i for i in range(len(genre))]))

def pos_process(corpus):
    global mood2idx, genre2idx
    corpus['genre'] = corpus['genre'].apply(lambda x: genre2idx[x])
    corpus['mood'] = corpus['mood'].apply(lambda x: mood2idx[x])
    return corpus

train_corpus = pos_process(train_corpus)
test_corpus = pos_process(test_corpus)
x_test = np.array(train_corpus['mood'])
y_test = np.array(test_corpus['mood'])

x = []
for i in range(len(train_corpus['sentence'])):
    if 'เพลง' in train_corpus['sentence'][i]:
        x.append(train_corpus['sentence'][i].replace('เพลง', ''))
    else:
        x.append(train_corpus['sentence'][i])
print(x)

x_corpus = []
for s in test_corpus['sentence']:
    if 'เพลง' in s:
        x_corpus.append(s.replace('เพลง', ''))
    else:
        x_corpus.append(s)
print(x_corpus)
    
    

['ชิว ฟังสบาย จังหวะเบา', 'ทำงานอยู่ สบายฟังชิวหน่อย', 'จัดมาสัก สบาย ฟังแล้วเพลิน', 'แบบ ฟังแล้วผ่อนคลาย สบายชิวไปวันนี้', 'เหนื่อยแล้วอยากพักผ่อนชิว มาฟังหย่อนใจ', 'ชิว กำลังสบายใจ ชิวชิว', 'แบบเพราะเพลินชิว', ' เบา ฟังสบายจิบกาแฟ', 'เพราะชิว ฟังสบาย', 'ชิวแบบนี้ พอฟังเพลิน เพราะ', 'อกหัก เศร้าบาดใจ ฟังช้า', 'แบบคนโดนเทหน่อย', 'เพิ่งโดนเพื่อนหักหลังมา ปลอบใจหน่อย', 'ตอนนี้เหงา อยากฟังอกหักหน่อย', 'ง่วง เหนื่อย อยากอยู่คนเดียว', 'โดนเท แก้เซ็ง', 'เศร้ามาก  เศร้า', 'แบบ เศร้ากว่านี้ไม่มีอีกแล้ว', 'เศร้า เสียใจมาก', 'เศร้าจับใจ โคตรเหงา', 'เบื่อจัง แก้เซ็งหน่อย', 'จัดเพลิน ฟังแล้วเฮฮาปาร์ตี้', ' rock หน่อย', 'แบบ สนุก ลุยบ้า', 'คึกคัก ฟังแล้วต้องเต้น', 'จัดมานึง มัน ร็อค', 'สนุก เฮฮา', 'แบบมัน แบบสนุกสนาน', 'สนุกสนาน ฟังสนุก', 'มัน ฟังแล้วต้องลุกมาเต้น', 'ที่ทำให้ใจสั่น', 'รักหน่อย', 'เพิ่งคุยกับแฟนมา อยากฟังชิวหน่อย', 'แอบชอบเธอจังเลย หน่อยน้า', 'รักหวานซึ้ง', 'รักแบบเขิน', 'วัยรุ่นใส วัยมีความรัก', 'เขิน ฟังแล้วฟินนนน รักเลย', 'รักใคร่', 'รัก', 'รักแบบหวาน ฟังเพราะ', 'แบบ ลูกทุ่ง หน่อ

In [638]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import linear_model

clf = linear_model.LogisticRegression(C=1e5)

vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x).toarray()
svd = TruncatedSVD(n_components=50, n_iter=10, random_state=42)
x_train = svd.fit_transform(x_train)
clf.fit(x_train, x_test) 
# vectorizer = CountVectorizer()
# x_train = vectorizer.fit_transform(train_corpus['sentence'])
# x_train = vectorizer.toarray()

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [639]:
x_test = vectorizer.transform(x_corpus).toarray()
x_test = svd.transform(x_test)

In [640]:
x_test.shape

(7, 49)

In [641]:
clf.predict(x_test)

array([4, 0, 0, 1, 1, 4, 2])

In [642]:
y_test

array([4, 0, 0, 1, 1, 4, 2])

In [643]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def evaluate(test_y, pred_y, labels=[i for i in range(0,10)]):
    f1 = f1_score(test_y, pred_y, average='macro', labels=labels)
    accuracy = accuracy_score(test_y, pred_y)
    precision = precision_score(test_y, pred_y, average='macro', labels=labels)
    recall = recall_score(test_y, pred_y, average='macro', labels=labels)
    return f1, accuracy, precision, recall

display(evaluate(y_test, clf.predict(x_test)))
# display(evaluate(class_z, pred_z))

/Users/fm_index/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/fm_index/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/fm_index/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/fm_index/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(0.40000000000000002, 1.0, 0.40000000000000002, 0.40000000000000002)

In [689]:
t = 'อยากเต้นเฮฮา'
if 'ชิว' in t:
    ans = np.array([3])
elif 'อกหัก' in t:
    ans = np.array([0])
elif 'สนุก' in t:
    ans = np.array([1])
else:
    t = vectorizer.transform([t]).toarray()
    t = svd.transform(t)
    ans = clf.predict(t)
print(ans)

[2]


In [690]:
mood2idx

{'all': 4, 'ชิว': 3, 'รัก': 2, 'สนุก': 1, 'เศร้า': 0}